In [1]:
using Knet;

## Features Dictionary Construction

This function constructs protein features dictionary. Protein features for the taining data set are stored in the file "yeast_feature_all.csv". 
The function returns a dictionary mapping a protein UniProt ID to its 1164-features vector
* ```featuresDict```    : The features dictionary

In [2]:
function constructFeatDict()
    # read features for all proteins
    f = open("yeast_feature_all.csv");
    lines = readlines(f);
    close(f);
    numberOfProteins = length(lines) 
    featureNames = String.(split(lines[1],",")); 
    d = length(featureNames) - 1 # number of features per protein
    proteins = lines[2:numberOfProteins];
    featuresDict = Dict{String,Any}()
    for p in proteins
        featureVect = String.(split(p, ","));
        featuresDict[featureVect[1]] = parse.(Float32, featureVect[2:d+1]) # featureVect[1] is the protein UniProt ID 
    end
    return featuresDict;
end;

In [ ]:
function constructFeatDictNegatome()
    # read features for all proteins
    f = open("yeast_feature_all.csv");
    lines = readlines(f);
    close(f);
    numberOfProteins = length(lines) 
    featureNames = String.(split(lines[1],",")); 
    d = length(featureNames) - 1 # number of features per protein
    proteins = lines[2:numberOfProteins];
    featuresDict = Dict{String,Any}()
    for p in proteins
        featureVect = String.(split(p, ","));
        featuresDict[featureVect[1]] = parse.(Float32, featureVect[2:d+1]) # featureVect[1] is the protein UniProt ID 
    end
    f = open("test_datasets/Negatome_features.csv");
    lines = readlines(f);
    close(f);
    numberOfProteins = length(lines) 
    featureNames = String.(split(lines[1],",")); 
    d = length(featureNames) - 1 # number of features per protein
    proteins = lines[2:numberOfProteins];
    for p in proteins
        featureVect = String.(split(p, ","));
        if("" in featureVect[2:d+1])
            continue
        end
        featuresDict[featureVect[1]] = parse.(Float32, featureVect[2:d+1]) # featureVect[1] is the protein UniProt ID 
    end
    return featuresDict;
end;

## Loading Data

This function loads the training data with its labels 
* ```featuresDict```    : The features dictionary which maps the protein ID to its features vector

The function returns two arrays, the data and the labels. 
In order to construct the data matrix, the following steps are executed:
* Protein pairs (represented by their UniProt IDs are read from the file "yeast_protein_pair.csv".
* A feature vector for each partner in the protein pair is extracted from ```featuresDict``` 
* The resulting two feature vectors are concatenated to form a new combined 2328-elements features vector
* Combined vectors for all pairs in the dataset are concatenated vertically to for the 65852x2328 data matrix ```concatAB```. 
* Labels for the protein pairs are read and returned as ```ygold```


In [3]:
function loaddata(featuresDict)
    f = open("yeast_protein_pair.csv")
    lines = readlines(f);
    close(f)
    n = length(lines); # number of samples/ protein pairs
    samples = lines[2:end - 1];
    concatAB1 = []
    ygold = Array{UInt8,1}(length(samples));
    i = 1;
    for s in samples
        s = String.(split(s, ","));
        push!(concatAB1, hcat(reshape(mat(featuresDict[s[1]]), 1, 1164), reshape(mat(featuresDict[s[2]]), 1, 1164)))
        label = parse(Int64, s[3]);
        ygold[i] = convert(UInt8, label + 1);
        i += 1;
    end
    concatAB = vcat(map(Atype, concatAB1)...)
    return concatAB, ygold
end;

In [ ]:
function loaddataNegatome(featuresDict)
    f = open("yeast_protein_pair.csv")
    lines = readlines(f);
    close(f)
    f = open("test_datasets/Negatome_pairs.csv")
    negsamples = readlines(f);
    close(f)
    n = length(lines); # number of samples/ protein pairs
    samples = lines[2:end - 1];
    possamples = samples[1:17257]
    concatAB1 = []
    ygold = Array{UInt8,1}(length(possamples) + length(negsamples));
    i = 1;
    for s in possamples
        s = String.(split(s, ","));
        push!(concatAB1, hcat(reshape(mat(featuresDict[s[1]]), 1, 1164), reshape(mat(featuresDict[s[2]]), 1, 1164)))
        ygold[i] = 0x02;
        i += 1;
    end
    
    
    n = length(negsamples); # number of samples/ protein pairs
    for s in negsamples
        s = String.(split(s, "\t"));
        if s[1] in keys(featuresDict) && s[2] in keys(featuresDict) 
            push!(concatAB1, hcat(reshape(mat(featuresDict[s[1]]), 1, 1164), reshape(mat(featuresDict[s[2]]), 1, 1164)))
            ygold[i] = 0x01;
            i += 1;
        end
    end
    
    concatAB = vcat(map(Atype, concatAB1)...)
    return concatAB, ygold
end;

## Initializing the model

This function initializes weights and bias for the model.
* ```h...```    : elements representing the number of inputs to the NN, the number of hidden units in each hidden layer and finally the number of outputs


In [4]:
function winit(h...)
    w = Any[]
    for i=2:length(h)
        push!(w, 0.01*randn(h[i],h[i-1]))
        push!(w, zeros(h[i],1))
    end
    map(Atype, w)
end;

## Preparing trn/dev/tst datasets
This function divides the total datset into trn/dev/tst sets using the ```trnper```, ```devper```, ```tstper```percentages. Before division, the negative subset to be used is picked randomly from the negative dataset to preserve a 1:1 positive:negative ratio.

* ```data```    : data matrix where each row represents a sample in the dataset
* ```ygold```    : labels of the dataset where 0x01 means the protein pair do not interact and 0x02 mean the pair interacts. 
* ```trnper```    : the percentage of the total dataset to be considered as training set
* ```devper```    : the percentage of the total dataset to be considered as development set
* ```tstper```    : the percentage of the total dataset to be considered as test set 
* ```batchsize``` : the number of samples to put in a minibatch

In [27]:
function dividedataset(data, ygold, trnper, devper, tstper; batchsize=64, dev=true) # 0.58, 0.17, 0.25
    nopos = 17257 # number of total positive samples
    
    # construct a 1:1 ratio of positive and negative samples as the data set
    posdata = data[1 : nopos, :];
    posygold = ygold[1 : nopos];
    negdata = data[nopos + 1 : end, :];
    negygold = ygold[nopos + 1 : end];
    
    # pick nopos negative samples randomly
    indneg = randperm(nopos)
    data = vcat(posdata, negdata[indneg[1:end],:])
    ygold = vcat(posygold, negygold[indneg[1:end]])
    
    nosamples = size(data,1)
    notst = Int(floor(tstper * nosamples))
    notrn = Int(floor(trnper * nosamples))
    nodev = nosamples - notrn - notst
    ind = randperm(nosamples)
    
    xtrn = data[ind[1:notrn],:];
    ytrn = ygold[ind[1:notrn]];
    
    xtst = data[ind[notrn+1:notrn+notst], :];
    ytst = ygold[ind[notrn+1:notrn+notst]];
    
    dtrn = minibatchi(xtrn',ytrn,batchsize);
    dtst = minibatchi(xtst',ytst,batchsize);
    
    if (dev)
        xdev = data[ind[notrn+1:notrn+nodev], :];
        ydev = ygold[ind[notrn+1:notrn+nodev]];
        ddev = minibatchi(xdev',ydev,batchsize);
        return dtrn, ddev, dtst
    else
        return dtrn, dtst
    end
    
end;

In [ ]:
function dividedatasetNegatome(data, ygold, trnper, devper, tstper; batchsize=64, dev=true) # 0.58, 0.17, 0.25
    nopos = 17257 # number of total positive samples
    noneg = 6222
    # construct a 1:1 ratio of positive and negative samples as the data set
    posdata = data[1 : nopos, :];
    posygold = ygold[1 : nopos];
    negdata = data[nopos + 1 : end, :];
    negygold = ygold[nopos + 1 : end];
    
    # pick noneg positive samples randomly
    indneg = randperm(noneg)
    data = vcat(posdata[indneg[1:end],:], negdata)
    ygold = vcat(posygold[indneg[1:end]], negygold)
    println(summary(data))
    nosamples = size(data,1)
    notst = Int(floor(tstper * nosamples))
    notrn = Int(floor(trnper * nosamples))
    nodev = nosamples - notrn - notst
    ind = randperm(nosamples)
    
    xtrn = data[ind[1:notrn],:];
    ytrn = ygold[ind[1:notrn]];
    
    xtst = data[ind[notrn+1:notrn+notst], :];
    ytst = ygold[ind[notrn+1:notrn+notst]];
    
    dtrn = minibatchi(xtrn',ytrn,batchsize);
    dtst = minibatchi(xtst',ytst,batchsize);
    
    if (dev)
        xdev = data[ind[notrn+1:notrn+nodev], :];
        ydev = ygold[ind[notrn+1:notrn+nodev]];
        ddev = minibatchi(xdev',ydev,batchsize);
        return dtrn, ddev, dtst
    else
        return dtrn, dtst
    end
    
end;

In [ ]:
function trn_tst_split(data, ygold, trnper, tstper; batchsize=64)
    nosamples = size(data,1)
    notrn = Int(floor(trnper * nosamples))
    notst = nosamples - notrn
    ind = randperm(nosamples)
    
    xtrn = data[ind[1:notrn],:];
    ytrn = ygold[ind[1:notrn]];
    
    xtst = data[ind[notrn+1:notrn+notst], :];
    ytst = ygold[ind[notrn+1:notrn+notst]];
    
    dtrn = minibatchi(xtrn',ytrn,batchsize);
    dtst = minibatchi(xtst',ytst,batchsize);
    
    return dtrn, dtst
end;

## MLP Forward Pass function
```predict``` function represents the forward pass of the MLP and is used to train DeepPPI-Con model

In [10]:
function predict(w,x; pdrop=(0,0))
    for i=1:2:length(w)
        x = dropout(x, pdrop[i==1 || i == length(w)-1?1:2])
        x = w[i]*x .+ w[i+1]
        if i < length(w)-1
            x = relu.(x)   ## apply RELU to all but the final layer's output                        
        end
    end
    return x
end;

```predictSep``` function represents the forward pass of the MLP and is used to train DeepPPI-Sep model
The data is first divided into a matrix ```xa``` having the features of the first partner in each protein pair from the total data set and another matrix ```xb``` containing the features of the second partner
Each of both matrices is input seperately into an MLP with seperate weights.
The output of both networks is then concatenated and input into another network which finally outputs a 2-D vector 

In [3]:
function predictSep(w, x; pdrop=(0,0), nofeat=1164)
    wa = w[1:6]
    wb = w[7:12]
    wm = w[13:end]
    xa = x[1:nofeat,:]
    xb = x[nofeat+1:end,:]
    
    for i=1:2:length(wa)
        xa = dropout(xa, pdrop[i==1?1:2])
        xa = wa[i]*xa .+ wa[i+1]
        xa = relu.(xa)                      
    end
    
    for i=1:2:length(wb)
        xb = dropout(xb, pdrop[i==1?1:2])
        xb = wb[i]*xb .+ wb[i+1]
        xb = relu.(xb)                         
    end
    
    xm = vcat(xa, xb)
    
    for i=1:2:length(wm)
        xm = dropout(xm, pdrop[i==1?2:1])
        xm = wm[i]*xm .+ wm[i+1]
        if i<length(wm)-1
            xm = relu.(xm)   ## apply RELU to all but the final layer's output                        
        end
    end
    return xm
end;

In [8]:
loss(w,x,ygold, predict; o...) = nll(predict(w,x; o...),ygold);
loss(w, data, predict; o...) = mean(loss(w,x,y,predict; o...) for (x,y) in data);
zeroone(w,data,predict; o...) = 1 - accuracyi(w,data,predict);
lossgradient = grad(loss);
report(epoch)=println((:epoch,epoch,:trn,accuracyi(w,dtrn,predict),:dev,accuracyi(w,ddev,predict)));

## Model Training 
```train!``` function trains the model(w) with the optimizer(s) specified in ```optims``` and returns lists containing error and loss values for both trn and dev/tst datasets **note that this is commented out for model evaluation**

In [ ]:
function train!(w, optims,data,predict, ddev; epochs=100,lr=.5,o...)
    patience = 0
    bestscore = Inf
    bestw = Any[]
    bestep = 0
    #trnloss = Any[loss(w,data,predict)]
    #trnerr = Any[zeroone(w,data,predict)]
    #devloss = Any[loss(w,ddev,predict)]
    deverr = Any[zeroone(w,ddev,predict)]
    for epoch in 1:epochs
        if patience > 10
          print("Early stopping no progess for 10 epoch... with best score: "*string(bestscore)*" during epoch: "*string(bestep)*"\n")
          break
        end
        if(epoch % 10 == 0)
            println((:epoch,epoch,:trn,accuracyi(w,data,predict),:tst,accuracyi(w,ddev,predict)));
        end
        for (x,y) in data
            dw = lossgradient(w,x,y, predict; o...)
            #@show (map(vecnorm, dw))
            update!(w, dw, optims)
#             for i in 1:length(w)
#                w[i] -= lr * dw[i]
#             end
        end
        #push!(trnloss, loss(w,data,predict))
        #push!(trnerr, zeroone(w,data,predict))
        #push!(devloss, loss(w,ddev,predict))
        deverror = zeroone(w,ddev,predict)
        push!(deverr, deverror)
        if deverror < bestscore
          bestw = deepcopy(w)
          bestscore = deverror
          bestep = epoch
          patience = 0
        else
          patience += 1
        end
    end
    w = deepcopy(bestw)
    #return trnloss, trnerr , devloss, deverr
end;

In [ ]:
# Train model(w) with SGD and return a list containing w for every epoch
function trainSep!(w, optims,data,predict, ddev; epochs=100,lr=.01,o...) #, decay=1.0
    patience = 0
    bestscore = Inf
    bestw = Any[]
    bestep = 0
    #trnloss = Any[loss(w,data,predict)]
    #trnerr = Any[zeroone(w,data,predict)]
    #devloss = Any[loss(w,ddev,predict)]
    deverr = Any[zeroone(w,ddev,predict)]
    for epoch in 1:epochs
        if patience > 10
          print("Early stopping no progess for 10 epoch... with best score: "*string(bestscore)*"\n")
          break
        end
        for (x,y) in data
            dw = lossgradient(w,x,y, predict; o...)
            #@show (map(vecnorm, dw))
            update!(w, dw, optims)
        end
        #push!(trnloss, loss(w,data,predict))
        #push!(trnerr, zeroone(w,data,predict))
        #push!(devloss, loss(w,ddev,predict))
        deverror = zeroone(w,ddev,predict)
        push!(deverr, deverror)
        if deverror < bestscore
          bestw = deepcopy(w)
          bestscore = deverror
          bestep = epoch
          patience = 0
        else
          patience += 1
        end
        if(epoch % 10 == 0)
            println((:epoch,epoch,:trn,accuracyi(w,data,predict),:tst,accuracyi(w,ddev,predict)));
        end
    end
    w = deepcopy(bestw)
    #return trnloss, trnerr, devloss, deverr
end;

## Optimzation methods
This function constructs an optimizer object ```prms``` for a weight array ```ws``` using the passed optimization parameters

In [12]:
#Creates necessary parameters for each weight to use in the optimization
function params(ws; optim="Sgd", lr=0.01, gamma=0.95, eps=1e-6, rho=0.9, beta1=0.9, beta2=0.95)
    prms = Any[]

    for i=1:length(ws)
        w = ws[i]
        if optim == "Sgd"
            prm = Sgd(;lr=lr)
        elseif optim == "Momentum"
            prm = Momentum(lr=lr, gamma=gamma)
        elseif optim == "Nesterov"
            prm = Nesterov(lr=lr, gamma=gamma)
        elseif optim == "Adagrad"
            prm = Adagrad(lr=lr, eps=eps)
        elseif optim == "Adadelta"
            prm = Adadelta(lr=lr, rho=rho, eps=eps)
        elseif optim == "Rmsprop"
            prm = Rmsprop(lr=lr, rho=rho, eps=eps)
        elseif optim == "Adam"
            prm = Adam(lr=lr, beta1=beta1, beta2=beta2, eps=eps)
        else
            error("Unknown optimization method!")
        end
        push!(prms, prm)
    end

    return prms
end;

# Performance Metrics
## Accuracy
This function calculates the accuracy of the predections produced by a model by the following formula:

```Accuracy = (number of correct predection)/(number of samples)```

**accuracyi(ypred, ygold)**

* ```ypred```    : predections produced by the models, for each samples a 2 dimensional one-hot vector is used, (1,0; i.e. if the value of the first element is higher thean the second) means no interaction while (0,1) means an interaction is predicted. 
* ```ygold```    : labels of the dataset where 0x01 means the protein pair do not interact and 0x02 mean the pair interacts.

**accuracyi(w, data, predict)**

This function returns an average accuracy over all minibatches
* ```w```    : model weights
* ```data```    : minibatched dataset containing both data and labels
* ```predict```    : the model


In [ ]:
function accuracyi(ypred, ygold)
    count = 0
    for i in 1:size(ypred, 2)
        if((ypred[1,i] >= ypred[2,i] && ygold[i]==1) || (ypred[1,i] <= ypred[2,i] && ygold[i]==2))
            count +=1
        end
    end
    return count/size(ypred, 2);
end;

In [7]:
function accuracyi(w, data, predict)
    acc = 0;
    for (x, y) in data
        ypred = predict(w,x)
        acc += accuracyi(ypred, y) 
    end
    return acc/length(data)
end;

## Other performance metrics
This function caculates and returns the following performance metrics

* ```Accuracy``` = (tp + tn) / (tp + tn + fn + fp)
* ```Recall``` = tp / (tp + fn)
* ```Specifity``` = tn / (tn + fp)
* ```Precision``` = tp / (tp + fp)
* ```MCC``` = (tp.tn-fp.fn)/(sqrt((tp+fp)(tp+fn)(tn+fp)(tn+fn)))
* ```F1``` : A statistical measure of the test accuracy which is the weighted average of precision and recall

$$f1 = (2tp)/(2tp+fp+fn)$$
* ```NPV``` : Negative predictive value (NPV) = Σ True negative/ Σ Predicted negatives

$$npv = (tn)/(tn + fn)$$

In [14]:
function modelevaluation(w, data, pred; p=false)
    tp=tn=fp=fn= 0    

    for (x,y) in data
        ypred = pred(w, x)
        for i in 1:size(ypred, 2)
            if(ypred[1,i] >= ypred[2,i] && y[i]==1)
                tn += 1;
            elseif(ypred[1,i] <= ypred[2,i] && y[i]==2)
                tp += 1
            elseif(ypred[1,i] >= ypred[2,i] && y[i]==2)
                fp += 1
            else
                fn += 1
            end
        end
    end
    accuracy = (tp + tn) / (tp + tn + fn + fp+1e-06)
    recall = tp / (tp + fn +1e-06)
    specifity = tn / (tn + fp+1e-06)
    precision = tp / (tp + fp +1e-06)
    mcc = (tp*tn-fp*fn)/(sqrt((tp+fp)*(tp+fn)*(tn+fp)*(tn+fn))+1e-06)
    f1 = (tp*2)/(tp*2+fp+fn+1e-06)
    npv = (tn)/(tn + fn + 1e-06)
    if (p)
        println("TP: ", tp, " , TN: ", tn, " , FP: ", fp, " , FN: ", fn);
        println("Model evaluation:");
        println("Accuracy : ", accuracy);
        println("Precision : ", precision);
        println("NPV : ", npv);
        println("Sensitivity / Recall : ", recall);
        println("Specifity : ", specifity);
        println("MCC : ", mcc);
        println("F1 : ", f1);
    end
    
    return accuracy,recall,specifity,precision,mcc,f1,npv;
end;

## Data Minibatching & Normalization

In [88]:
# Input X matrix and gold labels Y
# Output list of minibatches (x, y)
function minibatchi(X, Y, batchsize)
    data = Any[] 
    meanx = mean(X,2);
    if(gpu() < 0)
        # CPU
        stdx=std(X,2);
    else
        # for GPU, manually calculate std for Knet arrays
        stdx= sqrt.(sum(abs2.(X.-meanx), 2)/(size(X,2) - 1));
    end
    X = (X.-meanx) ./ stdx;

    for i = 1:batchsize:size(X, 2)
        bl = min(i + batchsize - 1, size(X, 2))
        push!(data, (X[:, i:bl], Y[i:bl]))
    end
    return data
end;

## Loading model
This function is used to load the parameters of a pretrained model 

In [ ]:
function loadmodel(filename,  w_sizes; Atype=gpu() >= 0 ? KnetArray{Float32} : Array{Float32})
    ## weight sizes of DeepPPI-Sep model
    # w_sizes = Tuple{Int64,Int64}[(512, 1164) (512, 1) (256, 512) (256, 1) (128, 256) (128, 1) (512, 1164) (512, 1) (256, 512) (256, 1) (128, 256) (128, 1) (128, 256) (128, 1) (2, 128) (2, 1)]
    ## weight sizes of DeepPPI-Con model
    # w_sizes = Tuple{Int64,Int64}[(512, 1164) (512, 1) (256, 512) (256, 1) (128, 256) (128, 1) (128, 128) (128, 1) (2, 128) (2, 1)]
    temp = readdlm(filename);
    w = Any[]
    for i in 1:size(temp, 1)
        temp1 = reshape(temp[i, 1:w_sizes[i][1]*w_sizes[i][2]], w_sizes[i])
        push!(w, temp1)
    end
    w = map(Atype, w)
    return w
end;

In [87]:
Atype = gpu() >= 0 ? KnetArray{Float32} : Array{Float32};
setseed(1);

# number of input features per protein
NOINPUTS = 1164;
# number of input features for the protein pair
NOCONCAT = NOINPUTS * 2;
# output is a one-hot-vector 10 -> not interacting, 01 -> intracting
NOOUTPUTS = 2;

# the percentages used for evaluationg models in the paper
trnper = 0.75;
tstper = 0.25;
devper = 1 - trnper - tstper;

# the percentages used during training and model analysis
#trnper = 0.58;
#devper = 0.17;
#tstper = 1 - trnper - devper;

featuresDict = constructFeatDict();
concatAB, ygold = loaddata(featuresDict);

In [90]:
#BATCHSIZE = 64;
#dtrn, ddev, dtst = dividedataset(concatAB, ygold, trnper, devper, tstper; batchsize= BATCHSIZE);